<a href="https://colab.research.google.com/github/P3aar/Cartorioebac/blob/main/4_ValidacaoQualidadeDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install pyspark

In [17]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, regexp_extract

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
spark = SparkSession.builder.getOrCreate()

In [20]:
produtos = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/produtos.csv', header=True, inferSchema=True)
clientes = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/clientes.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/itens_pedido.csv', header=True, inferSchema=True)
pedidos = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/pedidos.csv', header=True, inferSchema=True)

In [24]:
itens_pedido = itens_pedido.withColumn('preco_valido', when(col('preco') > 0, 'Válido').otherwise('Inválido'))

itens_pedido.select('id_pedido', 'id_produto', 'preco', 'preco_valido').show()
itens_pedido.filter(col('preco_valido') == 'Inválido').show()

+--------------------+--------------------+------+------------+
|           id_pedido|          id_produto| preco|preco_valido|
+--------------------+--------------------+------+------------+
|00010242fe8c5a6d1...|4244733e06e7ecb49...|  58.9|      Válido|
|00018f77f2f0320c5...|e5f2d52b802189ee6...| 239.9|      Válido|
|000229ec398224ef6...|c777355d18b72b67a...| 199.0|      Válido|
|00024acbcdf0a6daa...|7634da152a4610f15...| 12.99|      Válido|
|00042b26cf59d7ce6...|ac6c3623068f30de0...| 199.9|      Válido|
|00048cc3ae777c65d...|ef92defde845ab845...|  21.9|      Válido|
|00054e8431b9d7675...|8d4f2bb7e93e6710a...|  19.9|      Válido|
|000576fe39319847c...|557d850972a7d6f79...| 810.0|      Válido|
|0005a1a1728c9d785...|310ae3c140ff94b03...|145.95|      Válido|
|0005f50442cb953dc...|4535b0e1091c278df...| 53.99|      Válido|
|00061f2a7bc09da83...|d63c1011f49d98b97...| 59.99|      Válido|
|00063b381e2406b52...|f177554ea93259a5b...|  45.0|      Válido|
|0006ec9db01a64e59...|99a4788cb24856965.

In [26]:
status_permitidos = ['created', 'approved', 'delivered', 'shipped', 'canceled', 'invoiced', 'processing']

pedidos = pedidos.withColumn('status_valido', when(col('status_pedido').isin(status_permitidos), 'Valido').otherwise('Invalido'))

pedidos.select('id_pedido', 'status_pedido', 'status_valido').show()

pedidos.filter(col('status_valido') == 'Invalido').show()

+--------------------+-------------+-------------+
|           id_pedido|status_pedido|status_valido|
+--------------------+-------------+-------------+
|e481f51cbdc54678b...|    delivered|       Valido|
|53cdb2fc8bc7dce0b...|    delivered|       Valido|
|47770eb9100c2d0c4...|    delivered|       Valido|
|949d5b44dbf5de918...|    delivered|       Valido|
|ad21c59c0840e6cb8...|    delivered|       Valido|
|a4591c265e18cb1dc...|    delivered|       Valido|
|136cce7faa42fdb2c...|     invoiced|       Valido|
|6514b8ad8028c9f2c...|    delivered|       Valido|
|76c6e866289321a7c...|    delivered|       Valido|
|e69bfb5eb88e0ed6a...|    delivered|       Valido|
|e6ce16cb79ec1d90b...|    delivered|       Valido|
|34513ce0c4fab462a...|    delivered|       Valido|
|82566a660a982b15f...|    delivered|       Valido|
|5ff96c15d0b717ac6...|    delivered|       Valido|
|432aaf21d85167c2c...|    delivered|       Valido|
|dcb36b511fcac050b...|    delivered|       Valido|
|403b97836b0c04a62...|    deliv

In [29]:
for coluna in clientes.columns:
  clientes.select(count(col(coluna)).alias(coluna)).show()

for coluna in clientes.columns:
  clientes.select(count(when(col(coluna).isNull(), coluna)).alias(coluna)).show()

+----------+
|id_cliente|
+----------+
|     99441|
+----------+

+----------------+
|id_unico_cliente|
+----------------+
|           99441|
+----------------+

+-----------+
|cep_cliente|
+-----------+
|      99441|
+-----------+

+--------------+
|cidade_cliente|
+--------------+
|         99441|
+--------------+

+--------------+
|estado_cliente|
+--------------+
|         99441|
+--------------+

+----------+
|id_cliente|
+----------+
|         0|
+----------+

+----------------+
|id_unico_cliente|
+----------------+
|               0|
+----------------+

+-----------+
|cep_cliente|
+-----------+
|          0|
+-----------+

+--------------+
|cidade_cliente|
+--------------+
|             0|
+--------------+

+--------------+
|estado_cliente|
+--------------+
|             0|
+--------------+



In [30]:
n_nulls_pedidos_df = pedidos.select([count(when(col(c).isNull(),c)).alias(c) for c in pedidos.columns])
n_nulls_pedidos_df.show()

+---------+----------+-------------+------------------+---------------------+-------------------------+--------------------+---------------------+-------------+
|id_pedido|id_cliente|status_pedido|data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|status_valido|
+---------+----------+-------------+------------------+---------------------+-------------------------+--------------------+---------------------+-------------+
|        0|         0|            0|                 0|                  160|                     1783|                2965|                    0|            0|
+---------+----------+-------------+------------------+---------------------+-------------------------+--------------------+---------------------+-------------+



In [32]:
pedidos_integridade_df = pedidos.join(clientes, pedidos.id_cliente == clientes.id_cliente, 'left_anti')
pedidos_integridade_df.select('id_pedido', 'id_cliente').show()

+---------+----------+
|id_pedido|id_cliente|
+---------+----------+
+---------+----------+



In [34]:
itens_integridade_df = itens_pedido.join(produtos, itens_pedido.id_produto == produtos.id_produto,'left_anti')
itens_integridade_df.select('id_pedido', 'id_produto').show()

+---------+----------+
|id_pedido|id_produto|
+---------+----------+
+---------+----------+



In [35]:
clientes_validacao_df = clientes.withColumn('cep_valido', when(col('cep_cliente').rlike(r'^\d{5}$'), 'valido').otherwise('invalido')) # https://regexr.com/
clientes_validacao_df.select('id_cliente', 'cep_cliente', 'cep_valido').show()


+--------------------+-----------+----------+
|          id_cliente|cep_cliente|cep_valido|
+--------------------+-----------+----------+
|06b8999e2fba1a1fb...|      14409|    valido|
|18955e83d337fd6b2...|       9790|  invalido|
|4e7b3e00288586ebd...|       1151|  invalido|
|b2b6027bc5c5109e5...|       8775|  invalido|
|4f2d8ab171c80ec83...|      13056|    valido|
|879864dab9bc30475...|      89254|    valido|
|fd826e7cf63160e53...|       4534|  invalido|
|5e274e7a0c3809e14...|      35182|    valido|
|5adf08e34b2e99398...|      81560|    valido|
|4b7139f34592b3a31...|      30575|    valido|
|9fb35e4ed6f0a14a4...|      39400|    valido|
|5aa9e4fdd4dfd2095...|      20231|    valido|
|b2d1536598b73a9ab...|      18682|    valido|
|eabebad39a88bb6f5...|       5704|  invalido|
|1f1c7bf1c9b041b29...|      95110|    valido|
|206f3129c0e4d7d0b...|      13412|    valido|
|a7c125a0a07b75146...|      22750|    valido|
|c5c61596a3b6bd0ce...|       7124|  invalido|
|9b8ce803689b3562d...|       5416|

In [36]:
produtos_validacao_df = produtos.withColumn('id_produto_valido', when(col('id_produto').rlike(r'^[a-f0-9](32)$'), 'valido').otherwise('Invalido'))
produtos_validacao_df.select('id_produto', 'id_produto_valido').show()

+--------------------+-----------------+
|          id_produto|id_produto_valido|
+--------------------+-----------------+
|1e9e8ef04dbcff454...|         Invalido|
|3aa071139cb16b67c...|         Invalido|
|96bd76ec8810374ed...|         Invalido|
|cef67bcfe19066a93...|         Invalido|
|9dc1a7de274444849...|         Invalido|
|41d3672d4792049fa...|         Invalido|
|732bd381ad09e530f...|         Invalido|
|2548af3e6e77a690c...|         Invalido|
|37cc742be07708b53...|         Invalido|
|8c92109888e8cdf9d...|         Invalido|
|14aa47b7fe5c25522...|         Invalido|
|03b63c5fc16691530...|         Invalido|
|cf55509ea8edaaac1...|         Invalido|
|7bb6f29c2be577161...|         Invalido|
|eb31436580a610f20...|         Invalido|
|3bb7f144022e67327...|         Invalido|
|6a2fb4dd53d2cdb88...|         Invalido|
|a1b71017a84f92fd8...|         Invalido|
|a0736b92e52f6cead...|         Invalido|
|f53103a77d9cf245e...|         Invalido|
+--------------------+-----------------+
only showing top

In [38]:
if 'email_cliente' in clientes.columns:
  clientes_verificar_email_df = clientes.withColumn('email_valido', when(col('email_cliente').contains('@'), 'Valido').otherwise('Invalido'))
  clientes_verificar_email_df.select('id_cliente', 'email_cliente', 'email_valido').show()